# *Pré-processamento*

Esta é uma etapa importante do processamento de dados, que envolve vários conceitos. No pré-processamento os dados podem ser "limpos", padronizados e transformados. Ainda no pré-processamento ocorre também a extração e seleção de características.

*Limpar* os dados se refere a remoção, ou correção, de dados que estejam imcompletos, corrompidos ou imprecisos.
*Padronizar* os dados se refere a remover dados *outliers*, deixar todas as possíveis medidas na mesma escala e unidade e normalizar os dados se necerrário.
*Transformar* os dados se refere a colocar os dados em um formato que favoreça futuras extrações de características ou análises.

As características a serem extraídas dos dados dependem da natureza do dado a ser analisado. Os dados EMG são sinais elétricos coletados dentro de um período de tempo, portanto são dados no domínio do tempo e características do domínio do tempo são extraídas dele. Porém também é possível transformar os dados EMG para o domínio da frequência e extrair características do domínio da frequência. Existem diversas características que podem ser extraídas de cada domínio, entretando nem todas elas serão relevantes. Cada problema se beneficia de características, ou combinações de características, diferentes. Por tanto é preciso que haja uma seleção de características para encontrar a combinação de características que trará melhor resultado na análise dos dados.

## Limpeza dos dados

Ao coletar dados EMG, estes tendem a vir com uma interferência da rede elétrica. A interferência é na frequência da rede elétrica. No Brasil a rede elétrica possui uma frequência de 60 Hz, gerando uma interferência de mesma frequência. Em outros países a frequênciada rede elétrica é de 50 Hz, gerando uma interferência nesta faixa. Essa interferência precisa ser removida dos dados, para realizar tal remoção utiliza-se um filtro *notch* da frequência da rede elétrica. A base da dados utilizada neste tutorial já passou por este processo antes de ser disponibilizada pelo autor, portanto não é necessário aplicar este filtro novamente sobre os dados aqui utilizados.

## Transformação de dados

As características normalmente são extraidas sobre pequenas porções de tamanho fixo dos dados, não no dado como um todo. Essas pequenas porções são chamadas de janelas. A técnica de se separar o dado em janelas se chama janela deslizante (*sliding window*). Uma boa prática é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações não sejam perdidas.

(TODO: Colocar ume xemplo com imagens aqui, para exemplificar melhor o conceito)

Na base de dados sendo utilizada neste tutorial, cada ensaio tem duração de 5 segundos. Se utilizadas janelas com tamanho de 250ms, resultará em 20 janelas. Ao aplicar uma sobreposição de ~128ms, ficamos com 41 janelas de ~122ms.

In [1]:
# Carregando dados
import numpy as np

data = np.load('./lib/data/converted/s1.npy')
print(f'{data.shape} - (classes, ensaios, canais, linhas)')

(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)


In [5]:
from scipy.signal import stft

step = 470
segment = 1024
chunks_time = None
chunks_fft = None

for s in range(1,10):
    if s != 3 and s != 7:
        n_win = int((data.shape[-1] - segment) / step) + 1
        ids = np.arange(n_win) * step
        chunks_time = np.array([data[:,:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 3, 0, 4)
        _, _, chunks_fft = stft(data, fs=4000, nperseg=1024, noverlap=512)
        chunks_fft = np.swapaxes(chunks_fft, 3, 4)

print('Formato (shape) dos dados depois da divisão de janelas')
print('Dominio do tempo: {} - (classes, ensaios, canais, janelas, linhas)'.format(chunks_time.shape))
print('Dominio da frequência:  {} - (classes, ensaios, canais, janelas, linhas)'.format(chunks_fft.shape))

Formato (shape) dos dados depois da divisão de janelas
Time domain: (10, 6, 2, 41, 1024) - (classes, ensaios, canais, janelas, linhas)
FFT domain:  (10, 6, 2, 41, 513) - (classes, ensaios, canais, janelas, linhas)


## Características

Uma característica é uma propriedade individual mensurável ou característica de um fenômeno que está sendo observado. Em EMG uma característica pode ser extraida no domínio do tempo ou no domínio da frequência. As características a seguir foram retiradas do artigo *EMG Feature Extraction for Tolerance of White Gaussian Noise* \[1\].

### Domínio do tempo

1. Willison Amplitude (WAMP)

    ![WAMP](lib/img/features/WAMP.png)

2. Root Mean Square (RMS)

    ![RMS](lib/img/features/RMS.png)

3. Waveform Length (WL)
    
    ![WL](lib/img/features/WL.png)

4. Zero Crossing (ZC)

    ![ZC](lib/img/features/ZC.png)

### Domínio da frequência

1. Auto Regressive (AR)

    ![AR](lib/img/features/AR.png)

2. Median Frequency (FMD)

    ![FMD](lib/img/features/FMD.png)

3. Mean Frequency (FMN)

    ![FMN](lib/img/features/FMN.png)

\[1\] Phinyomark, Angkoon & Limsakul, Chusak & Phukpattaranont, P.. (2008). EMG Feature Extraction for Tolerance of White Gaussian Noise.
[Disponível neste link](https://www.researchgate.net/publication/263765853_EMG_Feature_Extraction_for_Tolerance_of_White_Gaussian_Noise)


### Extraindo características

É necessário implementar as características, geralmente em formato de funções ou métodos, para que seja possível aplicar tais funções aos dados de entrada e obter as características resultantes. A seguir temos a implementação da característica Zero Crossing (ZC).

In [ ]:
def ZC(x):
    aux = []
    for i in range(1023):
        aux.append((-x[:,:,:,:,i] * x[:,:,:,:,i+1]))
    aux = np.array(aux)
    aux = aux.transpose(1, 2, 3, 4, 0)
    s = np.sum(aux, axis=-1)
    return s

**Desafio:** Implemente todas as características apresentadas neste tutorial em formato de funções. Sinta-se livre também para buscar e implementar características EMG além das apresentadas, citando as fontes de tais características.